In [6]:
import numpy as np

In [7]:
mass=28.0855000000 

In [9]:
with open('FORCE_CONSTANTS_2ND','r') as fc:
    lines=fc.readlines()
    natom=int(lines[0])

    nlineblock=4
    fc_all=np.zeros((natom,natom,3,3))
    start=1
    for i in range(natom):
        for j in range(natom):
            fc_block=lines[start+1:start+nlineblock]
            fc=np.loadtxt(fc_block)
            fc_all[i,j]=fc/mass



In [10]:
fc_all[0,0]

array([[17.41754259, -0.        , -0.        ],
       [-0.        , 17.41754259, -0.        ],
       [-0.        , -0.        , 17.41754259]])

In [13]:
from ase.io import read
atoms=read('data.vasp',format='vasp')

#all atom is in central region by definition
#here we assume number of ghost shell is 1

#construct the list of vector connecting between each atoms

dist=np.zeros((natom,natom,3))
for i in range(natom):
    for j in range(natom):
        rmin=np.zeros(3)
        rij=atoms.get_distance(i,j,vector=True)
        rij_mic=atoms.get_distance(i,j,mic=True,vector=True)
        if(np.linalg.norm(rij_mic)<np.linalg.norm(rij)):
            rmin=rij_mic
        else:
            rmin=rij
        dist[i,j]=rmin

In [14]:
dist

array([[[ 0.    ,  0.    ,  0.    ],
        [ 0.    , -2.725 , -2.725 ],
        [-2.725 ,  0.    , -2.725 ],
        [-2.725 , -2.725 ,  0.    ],
        [ 1.3625,  1.3625,  1.3625],
        [ 1.3625, -1.3625, -1.3625],
        [-1.3625,  1.3625, -1.3625],
        [-1.3625, -1.3625,  1.3625]],

       [[ 0.    ,  2.725 ,  2.725 ],
        [ 0.    ,  0.    ,  0.    ],
        [-2.725 ,  2.725 ,  0.    ],
        [-2.725 ,  0.    ,  2.725 ],
        [ 1.3625, -1.3625, -1.3625],
        [ 1.3625,  1.3625,  1.3625],
        [-1.3625, -1.3625,  1.3625],
        [-1.3625,  1.3625, -1.3625]],

       [[ 2.725 ,  0.    ,  2.725 ],
        [ 2.725 , -2.725 ,  0.    ],
        [ 0.    ,  0.    ,  0.    ],
        [ 0.    , -2.725 ,  2.725 ],
        [-1.3625,  1.3625, -1.3625],
        [-1.3625, -1.3625,  1.3625],
        [ 1.3625,  1.3625,  1.3625],
        [ 1.3625, -1.3625, -1.3625]],

       [[ 2.725 ,  2.725 ,  0.    ],
        [ 2.725 ,  0.    , -2.725 ],
        [ 0.    ,  2.725 , -2.72